# machine vision assignment 

please run task 1 , then task 2 and then task 3

# Task -1 

In [1]:
#assignment part 1- preprocessing  (8 points)
import cv2
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import random
import os

images_path="Assignment_MV_02_calibration/"
video_path = "Assignment_MV_02_video.mp4"
output_path="output/"

def part1_A_and_B(images_path):
    """extract and display the checker board corners"""
    #kmeans 
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    objp = np.zeros((5*7,3), np.float32)
    
    #mgrid returns transpose of the meshgrid
    objp[:,:2] = np.mgrid[0:7,0:5].T.reshape(-1,2)                       
    objpoints = [] #will be used later to store
    imgpoints = [] #will be used later to store
    
    # Load all calibration images with .png extension from this directory and save it into a list
    images = glob.glob(images_path+'*.png')
    
    #this list contains all the images in a list
    #print(len(images))
    
    # Extract and display the checkerboard images
    for i,image in enumerate(images):
        
        #read every image
        img = cv2.imread(image) #read a given image in the directory
        
        #convert to black and white image
        grey_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        Check_board = (7,5)
        # corners to subpixel accuracy in all images using the OpenCV calibration tools
        ret, corners = cv2.findChessboardCorners(grey_img, Check_board,None)
        
        #ret denotes a status boolean True or False
        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            
            # Finds edges in an image using the [Canny86] algorithm.
            # source - https://docs.opencv.org/2.4/modules/imgproc/doc/feature_detection.html?highlight=cornersubpix
            corners2 = cv2.cornerSubPix(grey_img,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners)
            
            # Draw and display the corners over the image using draw chessboard corners
            img = cv2.drawChessboardCorners(img, Check_board, corners2, ret)
            cv2.imshow('This is my image '+str(i)+" showcasing corners", img)
            cv2.waitKey()
            
    cv2.destroyAllWindows()
    
    #this is part B
    #==========================================================================
    #print(gray_image.shape[::-1])

    # Determine and output the camera calibration matrix 𝑲 using the OpenCV calibration tools 
    ret, k, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, grey_img.shape[::-1],None,None)
    print("calibration matrix 𝑲:")
    print("="*20)
    print(k) 
    print("="*20)
    return k # Returning the calib matrix 𝑲

    
# Task 1, Part C
def part1C(video_path,output_path):
    # Open it for processing.
    # Getting first frame from the video 
    vid = cv2.VideoCapture(video_path) 
    currentframe = 1
    
    #if directory does not exist make new directory else save in the output directory
    import os
    cwd=os.getcwd()
    #print(cwd)
    try:
        os.chdir(cwd+"/"+output_path) #navigate to the output dir if possible
        os.chdir("..") #go back one step
        cwd=os.getcwd()
    except:
        os.mkdir(cwd+"/"+output_path)
    while(True): 
        ret,frame = vid.read() 
        if ret:  
            img_path = output_path + str(currentframe) + '.jpg'
            cv2.imwrite(img_path, frame) 
            break #if you want to save more frames you can do it by removing the break
    #print(path)
    # Release all space and windows once done 
    vid.release() 
    cv2.destroyAllWindows() 
    
    #load the image saved in the directory and change the format to grey scale or black and white
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2GRAY)
    
    # Identify good features to track in the first frameusing the OpenCV feature extraction and tracking functions.
    #tomasi criteria
    corners = cv2.goodFeaturesToTrack(img, 200, 0.3, 7) #refer slides
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001) #just like part 1
    
    # Refine the feature point coordinates to sub-pixel accuracy
    p0 = cv2.cornerSubPix(img,np.float32(corners),(5,5),(-1,-1),criteria)
    
    #to view feature points on the original image load image from directory
    img_o = cv2.imread(img_path)
    for i,points in enumerate(p0):
        x,y = points[0]
        #we draw a circle on the img_o of 4 thickness
        cv2.circle(img_o,(x,y), 4, (128,255,0), 1)
    cv2.imshow("feature pts:", img_o)
    cv2.waitKey()
    cv2.destroyAllWindows()
    return img_o

# Task 1, Part D
def part1D(video_path):
    """KLT algorithm refer lab 6 for this"""
    cv2.namedWindow("camera")
    camera = cv2.VideoCapture(video_path)

    # initialise features to track
    while camera.isOpened():
        ret,img= camera.read()
        if ret:
            new_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)        
            conors = cv2.goodFeaturesToTrack(new_img, 200, 0.3, 7)   
            # Make sure to refine the feature point coordinates to sub-pixel accuracy
            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
            p0 = cv2.cornerSubPix(new_img, np.float32(conors), (5, 5), (-1, -1), criteria)                                 
            break    

    #firstFrame = img.copy()
    # initialise tracks
    index = np.arange(len(p0))
    tracks = {}
    for i in range(len(p0)):
        tracks[index[i]] = {0:p0[i]}
    frame = 0
    while camera.isOpened():
        ret,img= camera.read()         
        frame += 1
        if not ret:
            break

        old_img = new_img
        new_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)        

        # calculate optical flow
        # Use the OpenCV implementation of the KLT algorithm to track these features across the whole image sequence [1 point]
        if len(p0)>0: 
            p1, st, err  = cv2.calcOpticalFlowPyrLK(old_img, new_img, p0, None)                                    

            # visualise points
            for i in range(len(st)):
                if st[i]:
                    cv2.circle(img, (p1[i,0,0],p1[i,0,1]), 2, (0,0,255), 2)
                    cv2.line(img, (p0[i,0,0],p0[i,0,1]), (int(p0[i,0,0]+(p1[i][0,0]-p0[i,0,0])*5),int(p0[i,0,1]+(p1[i][0,1]-p0[i,0,1])*5)), (0,0,255), 2)            

            p0 = p1[st==1].reshape(-1,1,2)            
            index = index[st.flatten()==1]


        # refresh features, if too many lost
        if len(p0)<100:
            new_p0 = cv2.goodFeaturesToTrack(new_img, 200-len(p0), 0.3, 7)
            #  Make sure to refine the feature point coordinates to sub-pixel accuracy
            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
            new_p0 = cv2.cornerSubPix(new_img, np.float32(new_p0), (5, 5), (-1, -1), criteria)

            for i in range(len(new_p0)):
                if np.min(np.linalg.norm((p0 - new_p0[i]).reshape(len(p0),2),axis=1))>10:
                    p0 = np.append(p0,new_p0[i].reshape(-1,1,2),axis=0)
                    index = np.append(index,np.max(index)+1)

        # update tracks
        for i in range(len(p0)):
            if index[i] in tracks:
                tracks[index[i]][frame] = p0[i]
            else:
                tracks[index[i]] = {frame: p0[i]}

        # visualise last frames of active tracks
        for i in range(len(index)):
            for f in range(frame-20,frame):
                if (f in tracks[index[i]]) and (f+1 in tracks[index[i]]):
                    cv2.line(img,
                             (tracks[index[i]][f][0,0],tracks[index[i]][f][0,1]),
                             (tracks[index[i]][f+1][0,0],tracks[index[i]][f+1][0,1]), 
                             (0,255,0), 1)

        # cut tracks that are too long
        # for i in tracks:
        #     if len(tracks[i])>50:
        #         keys = list(tracks[i].keys())
        #         for j in keys:
        #             if j<=max(keys)-50:
        #                 del tracks[i][j]

        k = cv2.waitKey(1)
        if k%256 == 27:
            print("Escape hit, closing...")
            break

        cv2.imshow("camera", img)        
        
    camera.release()
    cv2.destroyWindow("camera")   

    
    #self.imgshow(firstFrame,name="first Frame")
    #self.imgshow(lastFrame,name = "last frame")

    return frame-1,tracks

#functions call
K=part1_A_and_B(images_path)
img=part1C(video_path,output_path)
part1D(video_path)


calibration matrix 𝑲:
[[994.93741656   0.         485.13219674]
 [  0.         953.82995312 286.16775088]
 [  0.           0.           1.        ]]


(30,
 {0: {0: array([[383.86606, 312.49954]], dtype=float32),
   1: array([[383.21573, 311.1503 ]], dtype=float32),
   2: array([[380.98535, 311.45343]], dtype=float32),
   3: array([[384.0333 , 309.32883]], dtype=float32),
   4: array([[381.1404 , 310.65186]], dtype=float32),
   5: array([[380.2603 , 309.57938]], dtype=float32),
   6: array([[379.3458, 310.3674]], dtype=float32),
   7: array([[382.3162 , 308.18417]], dtype=float32),
   8: array([[378.874 , 308.4101]], dtype=float32),
   9: array([[377.38788, 309.0577 ]], dtype=float32),
   10: array([[376.19684, 309.00284]], dtype=float32),
   11: array([[379.3519, 305.9617]], dtype=float32),
   12: array([[375.12436, 308.6071 ]], dtype=float32),
   13: array([[374.67502, 308.14142]], dtype=float32),
   14: array([[375.28055, 307.19073]], dtype=float32),
   15: array([[376.28433, 306.15442]], dtype=float32),
   16: array([[371.71667, 307.96146]], dtype=float32),
   17: array([[370.43536, 307.3574 ]], dtype=float32),
   18: array([[373

In [2]:
#assignment part 2- essential matrix

# Task 2

In [3]:
# Part 2 
video_path = "Assignment_MV_02_video.mp4"

def part_2_a(video_path):
    """extract and visualize feature tracks"""
    #internally call part 1D for tracks and frames
    frame,tracks=part1D(video_path)
    frame1 = 0
    frame2 = frame
    frames=[frame1,frame2] #this is a list of frame numbers boundary start to stop 
    
    #extract frames
    extraction = {} #to store will be used later on
    camera = cv2.VideoCapture(video_path) #start capturing frames from the video provided
    frame=0
    while camera.isOpened(): 
        ret,img= camera.read()
        if not ret: #check for the return flag if it is false then video had some problems
            break
        if frame in frames: #check the frame
            extraction[frame] = img
        frame += 1
        if frame>frame2:# if frame count is greater than last frame then break out
            break
    
    correspondences = [] #will be used later store
    for track in tracks: #for each track in total tracks
        if (frame1 in tracks[track]) and (frame2 in tracks[track]):
            x1 = [tracks[track][frame1][0, 0], tracks[track][frame1][0, 1], 1]
            x2 = [tracks[track][frame2][0, 0], tracks[track][frame2][0, 1], 1]
            correspondences.append((np.array(x1), np.array(x2)))
            #draw circles on the frame with lines to indicate the feature tracks
            #draw red and green circles to highlight
            cv2.circle(extraction[frame2],(x1[0],x1[1]),2,(0,0,255),2)
            cv2.circle(extraction[frame2],(x2[0],x2[1]),2,(0,255,0),2)
            #the line color is green
            cv2.line(extraction[frame2], (x1[0],x1[1]),
                     (x2[0],x2[1]),
                     (0, 255, 0), 1)

    # cv2.imshow("task2A_extraction[0]",extraction[frame1])
    cv2.imshow("feature tracks visualization",extraction[frame2]) #to see the tracks in the image 
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    #this denotes the size of the list
    print(len(correspondences))
    return correspondences

# Part 2 Task B
def part_2_b(correspondences):
    
    #calculating the mean and std deviation using inbuilt numpy functions
    mean_x1 = np.mean(correspondences,axis=0)[0]
    mean_x2 = np.mean(correspondences,axis=0)[1]
    stddev_x1 = np.std(correspondences,axis=0)[0]
    stddev_x2 = np.std(correspondences,axis=0)[1]
    
    #print means and std of last and first frame
    print("Mean of the first frame:",mean_x1)
    print("Mean of the last frame:",mean_x2)
    print("Standard Deviation of the first frame: ",stddev_x1)
    print("Standard Deviation of the last frame:",stddev_x2)
    
    T = np.array([[1 / stddev_x1[0], 0, -mean_x1[0]/stddev_x1[0]],
                  [0, 1 / stddev_x1[1], -mean_x1[1]/stddev_x1[1]],
                  [0, 0, 1]])
    T_dash = np.array([[1 / stddev_x2[0], 0, -mean_x2[0]/stddev_x2[0]],
                  [0, 1 / stddev_x2[1], -mean_x2[1]/stddev_x2[1]],
                  [0, 0, 1]])
    
    yi_x1_f = []
    yi_dash_x2_l = []
    for x1, x2 in correspondences:
        yi_x1_f.append(np.matmul(T, x1))
        yi_dash_x2_l.append(np.matmul(T_dash, x2))

    # print("YI",yi_x1_f)
    # print("YI_DASH",yi_dash_x2_l)
    # print(len(yi_x1_f),len(yi_dash_x2_l))
    return yi_x1_f,yi_dash_x2_l,T,T_dash

def part2_c_d_e_f_g(correspondences, yi, yi_dash, T, T_dash, img):
    best_outliers = len(correspondences) + 1
    best_error = 1e100
    best_H = np.eye(3) #this is the identity mmatrix of order 3
    for iteration in range(10000): #this denotes the 10k different random selections
        # Part 2 Task C
        samples_in = set(random.sample(range(len(correspondences)), 8)) #select unique samples
        samples_out = set(range(len(correspondences))).difference(samples_in) #select unique samples out

        A = np.zeros((0, 9)) #start with zeros
        # print(samples_in)
        for i in samples_in:
            Ai = np.kron(np.transpose(yi[i]), yi_dash[i])
            A = np.append(A, np.array([Ai]), axis=0)
            
        # Part 2 Task D 
        #singular valued decomposition
        U, S, V = np.linalg.svd(A)
        F = V[8, :].reshape(3, 3).T

        U, S, V = np.linalg.svd(F) 
        F = np.matmul(U, np.matmul(np.diag([S[0], S[1], 0]), V))
        F = np.matmul(T_dash.T, np.matmul(F, T)) #taking matrix multiplication
        # # Part 2 Task E

        c_x_x = np.array([[1, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]])
        
        count_outliers = 0 #used for counting outliers
        accumulate_error = 0
        for i in samples_out:
            x1, x2 = correspondences[i]
            gi = np.matmul(np.matmul(x2.T, F), x1)

            sigma_square = np.matmul(np.matmul(np.matmul(np.matmul(x2.T, F), c_x_x), F.T), x2)
            sigma_square += np.matmul(np.matmul(np.matmul(np.matmul(x1.T, F.T), c_x_x), F), x1)

            # Part 2 Task F
            T_i = np.square(gi) / sigma_square
            if T_i > 6.635:
                count_outliers += 1
                # Part 2 Task H_1
                cv2.circle(img, (int(x1[0]),int(x1[1])), 5, (255, 0, 0), -1)
                cv2.circle(img, (int(x2[0]), int(x2[1])), 5, (255, 0, 0), -1)
            else:
                # print(x1, x2)
                accumulate_error += T_i
                # Part 2 Task H_1
                cv2.circle(img, (int(x1[0]), int(x1[1])), 5, (0, 0, 255), -1)
                cv2.circle(img, (int(x2[0]), int(x2[1])), 5, (0, 0, 255), -1)

        if (count_outliers < best_outliers):
            best_error = accumulate_error
            best_outliers = count_outliers
            best_H = F
        elif (count_outliers == best_outliers):
            if (accumulate_error < best_error):
                best_error = accumulate_error
                best_outliers = count_outliers
                best_H = F

    cv2.imwrite('besth_.jpg', img)
    # print(best_H)
    return best_H,F

# Part 2 Task H_2
def part2_h(bestH,img):
    #internally call part 1D for tracks and frames
    frame,tracks=part1D(video_path)
    frame1 = 0
    frame2 = frame
    frame=[frame1,frame2]
    
    #calculating single valued decomposition of bestH
    #calculating epipoles
    U, S, V = np.linalg.svd(bestH)
    e1 = V[2, :]
    U, S, V = np.linalg.svd(bestH.T)
    e2 = V[2, :]
    
    #print the results
    print("results :")
    print(e1 / e1[2])
    print(e2 / e2[2])

#functions call
res=part_2_a(video_path)
yi,yi_dash,T,T_dash =part_2_b(res)
best_h,f=part2_c_d_e_f_g(res,yi,yi_dash,T,T_dash,img)
part2_h(best_h,img)

69
Mean of the first frame: [526.09884599 270.98316773   1.        ]
Mean of the last frame: [536.85093589 246.86237366   1.        ]
Standard Deviation of the first frame:  [154.81588579  67.8869812    0.        ]
Standard Deviation of the last frame: [243.08924062  89.62239282   0.        ]
results :
[434.14539754 322.79872856   1.        ]
[432.7902019  320.81340771   1.        ]


# Task 3

In [7]:
# Part 3
def part_3_a(F,K):
    E = np.matmul(np.matmul(K.T,F),K)
    print("Essential matrix E :",E)
    #makes sure we have positive determinant
    U, S, V = np.linalg.svd(E)
    mean_s = (S[0]+ S[1]) /2.0
    S[0] = mean_s
    S[1] = mean_s

    U,S,V = np.linalg.svd(E)
    if np.linalg.det(U) < 0:
        U[:,2] *= -1
    if np.linalg.det(V) < 0:
        V[2,:] *= -1
        
    return E,U,S,V

def part3_b(E,U,S,V):
    #use the last frame 
    lastfr,tracks=part1D(video_path)
    w = np.array([
            [0,-1,0],
            [1,0,0],
            [0,0,1]
            ])
    z = np.array([
                [0,1,0],
                [-1,0,0],
                [0,0,0]
                ])
    
    videoLengthInSec = lastfr/30
    speedPerSec = 50/3.6
    baseLineDistance = videoLengthInSec * speedPerSec

    print("Base", baseLineDistance)
    Splus = U@z@U.T
    SMinus = -Splus


    R1 = U@w.T@V
    R2 = U@w@V

    t1 = U[:,2] 
    t2 = -U[:,2]

    print("R1", R1)
    print("R2", R2)
    print("T1", t1)
    print("T2", t2)
    print("S+", Splus)
    print("S-", SMinus)
    
#functions call        
e,u,s,v=part_3_a(f,K)
part3_b(e,u,s,v)

Essential matrix E : [[-5.30986204 33.281171    0.24539434]
 [ 2.03325812 87.54592901  1.63035364]
 [ 2.30006822 -1.79002924  0.11079186]]
Base 13.88888888888889
R1 [[ 0.37620879  0.86973302 -0.31942357]
 [ 0.92575546 -0.33870977  0.16808487]
 [ 0.03799708 -0.35894312 -0.93258569]]
R2 [[-0.33237576 -0.85748866 -0.39273343]
 [-0.94009073  0.33470534  0.06482091]
 [ 0.07586678  0.39074996 -0.91736509]]
T1 [ 0.35680487 -0.11669044  0.92686225]
T2 [-0.35680487  0.11669044 -0.92686225]
S+ [[ 2.70274725e-18  9.26862249e-01  1.16690443e-01]
 [-9.26862249e-01  1.45411112e-17  3.56804866e-01]
 [-1.16690443e-01 -3.56804866e-01 -2.94734491e-19]]
S- [[-2.70274725e-18 -9.26862249e-01 -1.16690443e-01]
 [ 9.26862249e-01 -1.45411112e-17 -3.56804866e-01]
 [ 1.16690443e-01  3.56804866e-01  2.94734491e-19]]
